# Packages

In [1]:
using Pkg
Pkg.add("MLDatasets")
Pkg.add("Flux")

include("graph.jl")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


backward (generic function with 9 methods)

# Dataset

In [2]:
using MLDatasets
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :test
  features  =>    28×28×10000 Array{Float32, 3}
  targets   =>    10000-element Vector{Int64}

In [3]:
using Flux
function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

loader (generic function with 1 method)

In [4]:
train_loader = loader(train_data)
test_loader = loader(test_data)

10000-element DataLoader(::Tuple{Matrix{Float32}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, shuffle=true)
  with first element:
  (784×1 Matrix{Float32}, 10×1 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

# Implemented operations

## Multiplication

In [87]:
import Base: *
import LinearAlgebra: mul!

# x * y (aka matrix multiplication)
*(A::GraphNode, x::GraphNode) = BroadcastedOperator(mul!, A, x)
forward(::BroadcastedOperator{typeof(mul!)}, A, x) = return A * x
backward(::BroadcastedOperator{typeof(mul!)}, A, x, g) = tuple(g * x', A' * g)

# x .* y (element-wise multiplication)
Base.Broadcast.broadcasted(*, x::GraphNode, y::GraphNode) = BroadcastedOperator(*, x, y)
forward(::BroadcastedOperator{typeof(*)}, x, y) = return x .* y
backward(node::BroadcastedOperator{typeof(*)}, x, y, g) = let
    𝟏 = ones(length(node.output))
    Jx = diagm(y .* 𝟏)
    Jy = diagm(x .* 𝟏)
    tuple(Jx' * g, Jy' * g)
end

backward (generic function with 14 methods)

## Addition

In [6]:
Base.Broadcast.broadcasted(+, x::GraphNode, y::GraphNode) = BroadcastedOperator(+, x, y)
forward(::BroadcastedOperator{typeof(+)}, x, y) = return x .+ y
backward(::BroadcastedOperator{typeof(+)}, x, y, g) = let
    println("ADD")
    return tuple(g, g)
end

backward (generic function with 9 methods)

## Summation

In [38]:
import Base: sum
sum(x::GraphNode) = BroadcastedOperator(sum, x)
forward(::BroadcastedOperator{typeof(sum)}, x) = let
    println("SUM_FWD")
    display(x)
    return sum(x)
end
backward(::BroadcastedOperator{typeof(sum)}, x, g) = let
    println("SUM")
    println("x: ")
    display(x)
    𝟏 = ones(Float32, size(x))
    println("1:")
    display(𝟏)
    tuple(𝟏 .* g)
end

backward (generic function with 14 methods)

## Tanh

In [8]:
import Base: tanh

tanh(x::GraphNode) = ScalarOperator(tanh, x)
forward(::ScalarOperator{typeof(tanh)}, x) = return tanh.(x)
backward(::ScalarOperator{typeof(tanh)}, x, g) = let
    println("TANH")
    return tuple((1 - tanh(x)^2) * g)
end

backward (generic function with 10 methods)

## Log

In [9]:
import Base: log
Base.Broadcast.broadcasted(log, x::GraphNode) = BroadcastedOperator(log, x)
forward(::BroadcastedOperator{typeof(log)}, x) = return log.(x)
backward(::BroadcastedOperator{typeof(log)}, x, g) = let
    println("LOG")
    tuple(g ./ x)
end

backward (generic function with 11 methods)

## Subtraction

In [10]:
import Base: -
-(x::GraphNode) = ScalarOperator(-, x)
forward(::ScalarOperator{typeof(-)}, x) = return .-x
backward(::ScalarOperator{typeof(-)}, x, g) = tuple(-g)

Base.Broadcast.broadcasted(-, x::GraphNode, y::GraphNode) = BroadcastedOperator(-, x, y)
forward(::BroadcastedOperator{typeof(-)}, x, y) = return x .- y
backward(::BroadcastedOperator{typeof(-)}, x, y, g) = let
    println("SUB")
    tuple(g,-g)
end

backward (generic function with 12 methods)

## Exp

In [11]:
import Base: exp 
Base.Broadcast.broadcasted(exp, x::GraphNode) = BroadcastedOperator(exp, x)
forward(::BroadcastedOperator{typeof(exp)}, x) = return exp.(x)
backward(::BroadcastedOperator{typeof(exp)}, x, g) = let
    println("EXP")
    tuple(exp.(x) .* g)
end

backward (generic function with 13 methods)

## Softmax

In [12]:
Softmax(x::GraphNode) = BroadcastedOperator(Softmax, x)
forward(::BroadcastedOperator{typeof(Softmax)}, x) = return exp.(x) ./ sum(exp.(x))
backward(node::BroadcastedOperator{typeof(Softmax)}, x, g) = let
    println("SOFTMAX")
    y = node.output
    J = diagm(y) .- y * y'
    tuple(J' * g)
end

backward (generic function with 14 methods)

## Division

In [13]:
Base.Broadcast.broadcasted(/, x::GraphNode, y::GraphNode) = BroadcastedOperator(/, x, y)
forward(::BroadcastedOperator{typeof(/)}, x, y) = return x ./ y
backward(node::BroadcastedOperator{typeof(/)}, x, y::Real, g) = let
    println("DIV")
    𝟏 = ones(Float32, length(node.output))
    Jx = diagm(𝟏 ./ y)
    Jy = (-x ./ y .^2)
    tuple(Jx' * g, Jy' * g)
end

backward (generic function with 14 methods)

# Net

In [14]:
INPUT_SIZE = 196
HIDDEN_SIZE = 64
OUTPUT_SIZE = 10

STEP_COUNT = 4

4

In [15]:
using LinearAlgebra

bound = 1/sqrt(HIDDEN_SIZE)  # read somewhere that this is a good way to init weights for tanh

Wi = Variable(bound .* rand(INPUT_SIZE, HIDDEN_SIZE), name="wi")
Wh = Variable(bound .* rand(HIDDEN_SIZE, HIDDEN_SIZE), name="wh")
Wo = Variable(randn(HIDDEN_SIZE, OUTPUT_SIZE), name="wo")

var wo
 ┣━ ^ 64×10 Matrix{Float64}
 ┗━ ∇ Nothing

In [77]:
function cross_entropy_loss(prediction, label)
    return sum(Constant(-1) .* label .* log.(prediction))
end

cross_entropy_loss (generic function with 1 method)

In [78]:
function net(sample, input_weights, hidden_weights, output_weights, label)
    i_1 = Variable(transpose(sample[1:196]), name="first_step_input")
    i_2 = Variable(transpose(sample[197:392]), name="second_step_input")
    i_3 = Variable(transpose(sample[393:588]), name="third_step_input")
    i_4 = Variable(transpose(sample[589:784]), name="fourth_step_input")

    s_1 = tanh(i_1 * Wi)
    s_1.name = "s_1"
    s_2 = tanh(i_2 * Wi .+ s_1 * Wh)
    s_2.name = "s_2"
    s_3 = tanh(i_3 * Wi .+ s_2 * Wh)
    s_3.name = "s_3"
    s_4 = tanh(i_4 * Wi .+ s_3 * Wh)
    s_4.name = "s_4"
    prediction = Softmax(s_4 * Wo)
    prediction.name = "prediction"

    E = cross_entropy_loss(prediction, Variable(label'))
    E.name = "loss"

    return topological_sort(E)
end

for (s, l) in test_loader
    graph = net(s, Wi, Wh, Wo, l)
    forward!(graph)
    backward!(graph)
end

xy MUL x:


-1

xy MUL y:


1×10 adjoint(OneHotMatrix(::Vector{UInt32})) with eltype Bool:
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

Ax MUL A:


...

xy MUL x:


1×10 Matrix{Int64}:
 0  0  -1  0  0  0  0  0  0  0

xy MUL y:


1×10 Matrix{Float64}:
 -14.6188  -15.4004  -3.57505  -10.3034  …  -9.56071  -15.1188  -0.894974

LoadError: DimensionMismatch: matrix A has dimensions (1,10), matrix B has dimensions (1,10)

In [89]:
forward!(topological_sort(Variable([1 2]) * Variable([1 2])))

LoadError: DimensionMismatch: matrix A has dimensions (1,2), matrix B has dimensions (1,2)